# Random-number simulations using NumPy

This notebook will illustrate the mechanics of solving a performing random-number simulations using the mechanics of NumPy we have already seen.  We will do two examples -- one sampling from a continuous distribution, one from a discrete distribution -- and see how to make a lot of headway with very few lines of code.

The examples can be found in the accompanying file `orf409_two_simulations.pdf`.

In the process, we'll encounter and use some useful functions from the `numpy.random` module.  The following link has a useful summary of the routines and functions found in `numpy.random`:

[https://docs.scipy.org/doc/numpy-1.14.0/reference/routines.random.html](https://docs.scipy.org/doc/numpy-1.14.0/reference/routines.random.html)

**NOTE:**  More recent versions of NumPy have adopted a somewhat different interface for generating random numbers than what is detailed in this notebook.  The main reason for those enhancements is performance and not user-friendliness.  So since the older interface is still supported (albeit deprecated), that is what you see in this notebook.  Unless you're running a crazy simulation on a cluster, you're better off with what's below, especially if you have limted prior experience with NumPy and Python.

## Setup -- importing numpy & seeding the random number generator

Before we start, we need to import numpy into this notebook.

In [ ]:
import numpy as np

Before each example, we should also set up a **seed** for the random number generator...

Computers don't *actually* generate random numbers.  Rather, they generate pseudo-random numbers that, in principle, can be reproduced exactly *provided* you prime the pseudorandom-number generator with the same **seed** each time.

Just so we are all getting the same results in our various notebooks, let's all use the same seed and set it up at the start of each problem (using a seed is good habit to get into, for personal debugging purposes).  The easiest thing to use as a seed is any biggish integer.  We'll do this at the start of each example problem.

## Simulation 1

This problem asks us to draw random values uniformly from the interval $[0,1)$ until the running sum exceeds 1.  Call $N$ the number of values we had to generate until we reached a sum above 1. Repeat this experiment 100 total times and average all the values of $N$ to get an estimate of $E[N]$.

Then repeat this experiment 1K times and 10K times and extrapolate to get an increasingly robust estimate for $E[N]$.

### Strategy
How should we implement this in code?

One strategy is to write a loop -- generate one random number at a time, keep a running sum, record how many values it takes to exceed 1, and then start the loop anew, looping over *that* inner loop 100 times (or 1K times or 10K times).

This would work, but it would be a lot of code.  Instead, we can lighten our coding burden by leveraging some obvious (and a few less obvious) features of numpy...

`np.random.random()` can be a given a single scalar argument (indicating the number of values to choose) or a tuple (indicating a shape if you want an array with more than 1 dimension).  Now, realistically, how many values do suspect we'd need to draw before we got a sum that exceeded 1? I'd say 1000 is probably fairly safe (and if it turns out not to be, we can come back and increase that value).

So why don't we just generate a 2D array of random values:  10K rows (each row is one "run" of the experiment), and 1000 columns (i.e. 1000 random values drawn in each experiment). Then, along each row, we can perform a `cumsum()` (which gives a running total) and pick out in each row the index where the running total has exceeded 1 (given how `cumsum()` works, i.e. that the entry in the result with index $N$ is the result of adding the first $N+1$ numbers in the original array, we'll need to add 1 to our index results to get the right value of $N$ in each experiment).

What we'll have by the end is an array of 10K $N$ values.  To estimate $E[N]$ using only 100 values, we can just slice, and average the first 100.  Ditto for the first 1000.

It turns aout that all of this can be done in very lines of numpy, with no loops anywhere.

Is it wasteful, in the sense that we're generating far more random values than we'll need?  Probably, but so what?  We're talking about 10 million floating point numbers.  That's maybe 100MB of RAM occupied and a few seconds of computation time.  And if it's really too much, we'll break up the problem and do 10 successive batches of 1000 runs.

The code is just a means to an end here.  Keep it simple, and let the computer do the work.

Let's see this in action...

In [ ]:
# Set random seed for the rest of this problem
np.random.seed(47404)

In [ ]:
# Make an array of 10K "runs" of 1000 random values each, from [0,1).
randvals = np.random.random((10000,1000))
randvals  # This line here just to display the output, so you see what's happening

In [ ]:
# For each row (meaning, fixing the axis-0 value and moving "along" axis-1), perform a cumsum().
# Neat fact -- when dealing with a 2D array, you can give aggregators like cumsum() an arugment axis=N to indicate
# which axis to move along as it aggregates.
running_sums = randvals.cumsum(axis=1)

In [ ]:
# Now, we need to locate the index along each row or running sums where the value in the cumsum() first exceeds 1.0.
# Here's a tricky way to achieve this...
# Recall that boolean expressions (e.g. running_sums > 1.0) applied to arrays return an array with the same shape,
# and with every entry True or False, depending on whether the condition held there or not.  So
# running_sums > 5
# is itself an array (of True/False values), and it looks like this:
running_sums > 1.0

In [ ]:
# Great.  In each row, we'd like to know where the first True is.  How do we do that?
# Fun fact: if you tell Python to interpret True & False numerically, it interprets True as 1 and False as 0.  Viewed
# this way, each row contains only the numbers 1 and 0.  So the max value along each row happens to be 1.  Thus,
# if we ran `argmax()` along each row, it would give us the index of one of the 1s.  Which 1?
# Ah... if you look at the help notes on np.argmax here:
# https://numpy.org/doc/stable/reference/generated/numpy.argmax.html
# you'll see under "Notes" that it says: "In case of multiple occurrences of the maximum values, the indices
# corresponding to the first occurrence are returned."

# That's exactly what we want!  Let's do it (argmax can also take that axis=1 argument, just like cumsum):
Nvals = np.argmax(running_sums > 1.0, axis=1)
print(Nvals)

In [ ]:
# Voila.  There are the 10K N values!

In [ ]:
# Now we need to get the avg of just the first 100, then first 1000, then all 10,000 N values.
# Slicing to the rescue
print(Nvals[:100].mean(), Nvals[:1000].mean(), Nvals.mean())

## Simulation 2

This problem asks us to simulate a weekly lottery.  Each week, five integers between 1 and 59 (inclusive) are drawn at random without replacement.  The first part of the problem asks us to write Python code to simulate this and to make sure we can calculate:

* $A(n)$ -- the number of times any given number $n$ is drawn  during the year; and
* $M$ -- the maximum number of times *any* number at all appears.

How should we go about simulating this lottery?


### Strategy

First, let's survey what is at our disposal.  It takes a little reading and digging, but the best option is `np.random.choice()`.  Check out its documentation here:

[https://docs.scipy.org/doc//numpy-1.10.4/reference/generated/numpy.random.choice.html](https://docs.scipy.org/doc//numpy-1.10.4/reference/generated/numpy.random.choice.html)

It has an optional argument `replace=True/False` to indicate whether the draws should be with or without replacement.  It needs to be given the integer array to draw from as an argument, but that's straightforward -- we can build an integer array with `arange()`.

Alternately, we could just do a *permutation* of the values from 1 to 59 and slice out the first 5 from the area. This might be the better choice, since the documentation for `np.random.permutation()` states that "If \[the array you feed it\] is a multi-dimensional array, it is only shuffled along its first index.", i.e. along axis-0.

Great.  So the whole year's lottery amounts to:

* making a 52 x 59 array -- each row represents a week
* applying `permutation()` along each row
* slicing out the first 5 columns to end with a 52 x 5 matrix

With a combination of a list comprehension, `random.permutation()`, and slicing, this is quick (remember, better to reuse tools than to reinvent the wheel):

In [ ]:
# Set random seed for the rest of this problem
np.random.seed(47404)

In [ ]:
# Make a 2D array by passing np.array() a list of 52 1d-arrays (which are each permutations of 1-59), and then
# slice it.  Remember -- arange(1,60) will start at 1 and end at 59
lottery_1yr = np.array([np.random.permutation(np.arange(1,60))[:5] for i in range(52)])
lottery_1yr  # Just here so you can see the results

Now how should our function A(n) work?  Well, we can use the whole numpy-True/False-array trick again...

Remember, something like `lottery_1yr == 42` is itself an *array* of True/False values.  The Trues are treated as 1, the Falses as 0.  So we can get a count of the number of 42s by running `sum()` on the array `lottery_1yr == 42`.

To get the number of occurrences of all the numbers from 1 to 59, use a list comprehension:

In [ ]:
num_occurrences = np.array([np.sum(lottery_1yr == i) for i in range(1,60)])
num_occurrences

And of course, to get the maximum value of *any* occurrence, just take the `max()` of that array:

In [ ]:
num_occurrences.max()

Working out the remainder of the problem is left as an exercise.

## Moral of the story

Before you write a lot of code, see if there are existing tools and constructs you can cobble together to do what you want, especially in a course where the coding is means and not the ends.